In [1]:
import os 
os.chdir("../")

In [2]:
%pwd

'c:\\datascience\\mypractice\\Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    dataset_name: str
    local_data_file: Path

# these are the thing we are returing as an entity

In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories


In [5]:
CONFIG_FILE_PATH, PARAMS_FILE_PATH

(WindowsPath('config/config.yaml'), WindowsPath('params.yaml'))

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            dataset_name=config.dataset_name,
            local_data_file=config.local_data_file
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size

In [15]:
from datasets import load_dataset

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_dataset(self):
        if not self.config.dataset_name:
            logger.error("Dataset name not provided.")
            return None
        
        try:
            logger.info(f"Downloading dataset '{self.config.dataset_name}' from Hugging Face...")
            dataset = load_dataset(self.config.dataset_name)
            logger.info(f"Dataset '{self.config.dataset_name}' downloaded successfully.")

            # Save the dataset to the specified directory
            dataset.save_to_disk(self.config.local_data_file)

            logger.info(f"Dataset saved to {self.config.local_data_file}")
            return dataset
        except Exception as e:
            logger.error(f"Failed to download dataset '{self.config.dataset_name}'. Error: {str(e)}")
            return None


In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_dataset()
except Exception as e:
    raise e

[ 2024-04-26 02:39:23,220 ] 31 TextSummarizer.logging - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-04-26 02:39:23,221 ] 31 TextSummarizer.logging - INFO - yaml file: params.yaml loaded successfully
[ 2024-04-26 02:39:23,222 ] 53 TextSummarizer.logging - INFO - created directory at: artifacts
[ 2024-04-26 02:39:23,222 ] 53 TextSummarizer.logging - INFO - created directory at: artifacts/data_ingestion
[ 2024-04-26 02:39:23,223 ] 13 TextSummarizer.logging - INFO - Downloading dataset 'samsum' from Hugging Face...
[ 2024-04-26 02:39:32,566 ] 15 TextSummarizer.logging - INFO - Dataset 'samsum' downloaded successfully.


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 272643.09 examples/s]

[ 2024-04-26 02:39:32,590 ] 20 TextSummarizer.logging - INFO - Dataset saved to artifacts/data_ingestion/samsum
